In [45]:
from queries import *
from pathlib import Path
from pandas_profiling import ProfileReport
from sql_metadata import Parser
import sqlparse
import sqlglot
import re
from pprint import pprint

In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

In [3]:
engine = connect_sqlserver_windowsauth('bidatacentre','master')
db_df = fetch_all_dbs(engine)


In [4]:
def markdown_table(in_df):
    header = ""
    line2 = ""
    data = ""
    for col in in_df.columns:
        header = header + f"| {col} "
        line2 = line2 + f"| --- "
    header = header + "|\n"
    line2 = line2 + "|\n"
    for row in in_df.itertuples():
        for col in range(1,len(row)):
            data = data + f"| {row[col]} "
        data = data + "|\n"
    table_string = header + line2 + data + "\n"
    return table_string

In [5]:
# Test code
deps = fetch_all_db_dependencies(engine)

In [6]:
deps


,DATABASE,referencing_schema_name,referencing_object_name,referencing_type_desc,referenced_server_name,referenced_db_name,referenced_schema_name,referenced_entity_name
0,APPS,dbo,AX_GL_BudgetDetails,VIEW,None,BI,dbo,AX_Budget_Trans
1,APPS,dbo,AX_GL_Master,VIEW,None,AX_SECURE_PRD_REPLICA,dbo,MAINACCOUNT
2,APPS,dbo,AX_GL_Master,VIEW,None,AX_SECURE_PRD_REPLICA,dbo,MAINACCOUNTCATEGORY
3,APPS,dbo,AX_GL_TransDetails,VIEW,None,BI,dbo,AX_GL_TransDetails
4,APPS,dbo,AX_PChemCustomer,VIEW,None,BI,dbo,AX_Customer_Master
...,...,...,...,...,...,...,...,...
41519,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectByWorkflowSubreportRS,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,TaskStatusTranslated
41520,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectByWorkflowSubreportRS,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,TaskTypesTranslated
41521,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectByWorkflowSubreportRS,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,WorkflowTasks
41522,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectList,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,WorkflowTasks


In [7]:
get_children(deps, 'BI', 'dbo', 'AX_Budget_Trans')

,DATABASE,referencing_schema_name,referencing_object_name,referencing_type_desc
0,APPS,dbo,AX_GL_BudgetDetails,VIEW
24974,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-01 Facility Roll Up Print Cop...,SQL_STORED_PROCEDURE
24975,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-01 Facility Roll Up Print Cop...,SQL_STORED_PROCEDURE
24979,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-01 Facility Roll Up Print Cop...,SQL_STORED_PROCEDURE
24980,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-02 PRD Summary Report - CAN__1,SQL_STORED_PROCEDURE
1217,BI,dbo,zPowerBI_PRDQForecast,VIEW
1455,BI,temp,/Report Parts/Admin Temp/PRD-FIN-XX PRD Summar...,SQL_STORED_PROCEDURE


## VIEWS

### Get all View definitions for this DB ('BI')

In [ ]:
engine = connect_sqlserver_windowsauth('bidatacentre','BI')
view_def = fetch_view_definitions(engine)

In [10]:
view_def

,name,definition
0,AX_FixedAsset_TransDetails,\r\n\r\n\r\n\r\n\r\nCREATE VIEW [dbo].[AX_Fixe...
1,zExcel_DPS_CustomerRevenue,\r\n\r\n\r\n\r\n\r\n\r\n/*********************...
2,AX_Inventory_Location,\r\n\r\n\r\n\r\n\r\nCREATE VIEW [dbo].[AX_Inve...
3,zPowerBI_AXRevenue_PRDTransWithDistance,\r\n\r\n\r\nCREATE VIEW [dbo].[zPowerBI_AXReve...
4,zPowerBI_PRDQForecast,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n/*****...
...,...,...
209,AX_Facility_UserAccess,\r\n/*****************************************...
210,AX_GL_Master,\r\n\r\nCREATE VIEW [dbo].[AX_GL_Master] AS \r...
211,AX_Project_Master,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...
212,AX_Vendor_Trans,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nCREATE VIEW [d...


In [42]:
# Get 1 definition to play with
defn = view_def['definition'][0]
defn

"\r\n\r\n\r\n\r\n\r\nCREATE VIEW [dbo].[AX_FixedAsset_TransDetails] AS \r\nSELECT\r\n\ttrans.TRANSDATE as AssetTransDate\r\n\t,trans.ASSETID as AssetID\r\n\t,trans.BOOKID as BookID\r\n\t,trans.TXT AS Description\r\n\t,trans.AMOUNTMST as AccountingCurrency\r\n\t,trans.CURRENCYCODE as CurrencyCode\r\n\t,trans.AMOUNTCUR as TransactionCurrency\r\n\t,trans.VOUCHER as Voucher\r\n\t,Trans.ASSETGROUP AS FixedAssetGroup\r\n\t,CASE WHEN trans.TRANSTYPE = 0 THEN 'None' \r\n\tWHEN trans.TRANSTYPE = 1 THEN 'Acquisition'\r\n\tWHEN trans.TRANSTYPE = 2 THEN 'Acquisition Adjustment'\r\n\tWHEN trans.TRANSTYPE = 3 THEN 'Depreciation'\r\n\tWHEN trans.TRANSTYPE = 4 THEN 'Depreciation Adjustment'\r\n\tWHEN trans.TRANSTYPE = 5 THEN 'Revaluation'\r\n\tWHEN trans.TRANSTYPE = 6 THEN 'WriteupAdjustment'\r\n\tWHEN trans.TRANSTYPE = 7 THEN 'WritedownAdjustment'\r\n\tWHEN trans.TRANSTYPE = 8 THEN 'Disposal-Sale'\r\n\tWHEN trans.TRANSTYPE = 9 THEN 'Disposal-Scrap'\r\n    ELSE CAST(trans.TRANSTYPE as varchar(2)) END 

### Strip off VIEW DDL with a regular expression

In [54]:
#  defn.replace("CREATE VIEW _anything_ AS ","")
new_defn = re.sub("CREATE VIEW (.*) AS ", "", defn)
new_defn

"\r\n\r\n\r\n\r\n\r\n\r\nSELECT\r\n\ttrans.TRANSDATE as AssetTransDate\r\n\t,trans.ASSETID as AssetID\r\n\t,trans.BOOKID as BookID\r\n\t,trans.TXT AS Description\r\n\t,trans.AMOUNTMST as AccountingCurrency\r\n\t,trans.CURRENCYCODE as CurrencyCode\r\n\t,trans.AMOUNTCUR as TransactionCurrency\r\n\t,trans.VOUCHER as Voucher\r\n\t,Trans.ASSETGROUP AS FixedAssetGroup\r\n\t,CASE WHEN trans.TRANSTYPE = 0 THEN 'None' \r\n\tWHEN trans.TRANSTYPE = 1 THEN 'Acquisition'\r\n\tWHEN trans.TRANSTYPE = 2 THEN 'Acquisition Adjustment'\r\n\tWHEN trans.TRANSTYPE = 3 THEN 'Depreciation'\r\n\tWHEN trans.TRANSTYPE = 4 THEN 'Depreciation Adjustment'\r\n\tWHEN trans.TRANSTYPE = 5 THEN 'Revaluation'\r\n\tWHEN trans.TRANSTYPE = 6 THEN 'WriteupAdjustment'\r\n\tWHEN trans.TRANSTYPE = 7 THEN 'WritedownAdjustment'\r\n\tWHEN trans.TRANSTYPE = 8 THEN 'Disposal-Sale'\r\n\tWHEN trans.TRANSTYPE = 9 THEN 'Disposal-Scrap'\r\n    ELSE CAST(trans.TRANSTYPE as varchar(2)) END as TransType\r\n\t,asset.name AS AssetName\r\n\t,t

### Cleanup SQL formatting for display to user

In [57]:
#stmts =  sqlparse.split(new_defn)
reformatted = sqlparse.format(new_defn, reindent=True, keyword_case='upper')
print(reformatted)


SELECT trans.TRANSDATE AS AssetTransDate ,
       trans.ASSETID AS AssetID ,
       trans.BOOKID AS BookID ,
       trans.TXT AS Description ,
       trans.AMOUNTMST AS AccountingCurrency ,
       trans.CURRENCYCODE AS CurrencyCode ,
       trans.AMOUNTCUR AS TransactionCurrency ,
       trans.VOUCHER AS Voucher ,
       Trans.ASSETGROUP AS FixedAssetGroup ,
       CASE
           WHEN trans.TRANSTYPE = 0 THEN 'None'
           WHEN trans.TRANSTYPE = 1 THEN 'Acquisition'
           WHEN trans.TRANSTYPE = 2 THEN 'Acquisition Adjustment'
           WHEN trans.TRANSTYPE = 3 THEN 'Depreciation'
           WHEN trans.TRANSTYPE = 4 THEN 'Depreciation Adjustment'
           WHEN trans.TRANSTYPE = 5 THEN 'Revaluation'
           WHEN trans.TRANSTYPE = 6 THEN 'WriteupAdjustment'
           WHEN trans.TRANSTYPE = 7 THEN 'WritedownAdjustment'
           WHEN trans.TRANSTYPE = 8 THEN 'Disposal-Sale'
           WHEN trans.TRANSTYPE = 9 THEN 'Disposal-Scrap'
           ELSE CAST(trans.TRANSTYPE AS 

### Show original table to table alias

In [66]:
t_list=[]
for table in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.Table):
    t_list.append(table)
ta_list=[]

for tablealias in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.TableAlias):
    ta_list.append(tablealias)

print(len(t_list), len(ta_list))

for t,ta in zip(t_list, ta_list):
    print(f"{t} ---> {ta}")

3 3
AX_SECURE_PRD_REPLICA.dbo.ASSETTRANS ---> trans
AX_SECURE_PRD_REPLICA.dbo.ASSETTABLE ---> asset
dbo.AX_Default_Dimensions ---> tdim


### Show original column to column alias

### *THIS DOES NOT WORK -- CASE STATEMENT IS MISSING SO EVERTHING BELOW IT IS MISALIGNED*

In [67]:
# DOES NOT WORK  --- LISTS HAVE DIFFERENT LEN ---
a_list = []
for column in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.Column):
  a_list.append(column.alias_or_name)
b_list = []
for select in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.Select):
  for projection in select.args["expressions"]:
    b_list.append(projection.alias_or_name)
print(f"COLUMNS: {len(a_list)}   ALIASES: {len(b_list)}")
print(a_list, "\n")
print(b_list, "\n")
print("\nWRONG DATA BELOW")
for a,b in zip(a_list, b_list):
    print(f"{a} ---> {b}")

COLUMNS: 39   ALIASES: 22
['TRANSDATE', 'ASSETID', 'BOOKID', 'TXT', 'AMOUNTMST', 'CURRENCYCODE', 'AMOUNTCUR', 'VOUCHER', 'ASSETGROUP', 'name', 'DATAAREAID', 'Division', 'LegalEntity', 'BusinessUnit', 'Location', 'WellType', 'COM', 'MAINTENANCEINFO1', 'DEFAULTDIMENSION', 'TRANSTYPE', 'DEFAULTDIMENSION', 'DEFAULTDIMENSION', 'TRANSTYPE', 'ASSETID', 'ASSETID', 'DATAAREAID', 'DATAAREAID', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE', 'TRANSTYPE'] 

['AssetTransDate', 'AssetID', 'BookID', 'Description', 'AccountingCurrency', 'CurrencyCode', 'TransactionCurrency', 'Voucher', 'FixedAssetGroup', 'TransType', 'AssetName', 'Company', 'TransDivision', 'TransLegalEntity', 'TransBusinessUnit', 'TransLocation', 'TransWellType', 'TransCom', 'UnitID', 'Dimension', 'TransTypeRaw', 'TransTypeSum'] 


WRONG DATA BELOW
TRANSDATE ---> AssetTransDate
ASSETID ---> AssetID
BOOKID ---> BookID
TXT ---> Description
AM

# GENERATE .md files for use in obsidian or wiki

In [63]:
# Generate base directories + include directories
DataCatalog_Path = Path("C:/obsidian/datacatalog/bidatacentre")
Include_Path = DataCatalog_Path / "include"

for db in list(db_df['name']):
    engine = connect_sqlserver_windowsauth('bidatacentre',db)
    # Table Details
    tab_det_df = fetch_table_details(engine)
    tab_dates = fetch_table_dates_and_rowcount(engine)
    # View Details

    # Get all DB Dependencies
    deps_df = fetch_all_db_dependencies(engine)

    for db_schema in list(tab_det_df['schema_name'].unique()):
        (Include_Path / db / db_schema).mkdir(parents=True, exist_ok=True)
        (DataCatalog_Path / db / db_schema).mkdir(parents=True, exist_ok=True)
        # drill down to individual schema / table and generate files.
        for table in list((tab_det_df.loc[tab_det_df['schema_name'] == db_schema])['table_name'].unique()):
            # Just the column details for this table
            tab_df = tab_det_df.loc[ (tab_det_df['schema_name'] == db_schema) & (tab_det_df['table_name'] == table) ]
            tab_df = tab_df[['column_name',
                             'data_type_ext',
                             'nullable',
                             'default_value',
                             'primary_key',
                             'foreign_key',
                             'unique_key',
                             'comments']]
            tab_df.rename(columns={'column_name': 'Column',
                     'data_type_ext': 'DataType',
                     'nullable': 'Nullable',
                     'default_value': 'Default',
                     'primary_key': 'IS_PK',
                     'foreign_key': 'IS_FK',
                     'unique_key': 'Unique',
                     'comments': 'Comments'}, inplace=True)

            tabfile = table + ".md"
            row = tab_dates.loc[(tab_dates['schema_name'] == db_schema) & (tab_dates['table_name'] == table)]
            with open ((DataCatalog_Path / db / db_schema / tabfile ), mode='w') as file:
                file.write("##### SUMMARY :\n\n")
                file.write(f"DB: **{db}**  SCHEMA: **{db_schema}**  OBJECT: **{table}** \n")
                file.write(f"Created: **{row.iloc[0]['created']}**  Modified: **{row.iloc[0]['last_modified']}**\n")
                file.write(f"Rows: **{row.iloc[0]['num_rows']}**\nComments: **{row.iloc[0]['comments']}**\n")
                file.write("\n\n")

                file.write("##### DEPENDENCIES :\n\n")
                file.write("Parents:\n")
                parents = get_parents(deps_df, db, db_schema, object)
                if len(parents) >= 1:
                    file.write(markdown_table(parents))
                else:
                    file.write("**None**")
                file.write("\n\n")

                file.write("Children:\n")
                children = get_children(deps_df, db, db_schema, object)
                if len(children) >= 1:
                    print(children)
                    file.write(markdown_table(children))
                else:
                    file.write("**None**")
                file.write("\n\n")

                file.write("##### COLUMN DEFINITIONS:\n\n")
                file.write(markdown_table(tab_df))
                file.write("\n\n")
                file.write("##### EXPLORATORY DATA ANALYSIS:\n\n")
                # Use actual data for tables with less than 1000 rows
                if row.iloc[0]['num_rows'] <= 1000:
                    rows = fetch_all_rows(engine, db_schema, table)
                    # Using pandas_profiling report for simple EDA
                    profile = ProfileReport(rows, title=table)
                    tabfile = f"{table}_profile.html"
                    filename = (Include_Path / db / db_schema / tabfile )
                    profile.to_file(filename)
                    file.write(f'<iframe src="{filename}" height=300 width=1000>\n\n')
                    file.write(f"Open in browser: ![[{tabfile}]]\n")
                    file.write("\n\n")
                else:
                    # Methods to sample a random percentage of a table in MS SQL Server
                    # https://docs.microsoft.com/en-us/previous-versions/sql/sql-server-2008-r2/ms189108(v=sql.105)?redirectedfrom=MSDN
                    #rows = fetch_sample_rows(engine, db_schema, table, sample_pct=10)

                    file.write("\n\n")

                file.write("##### USER NOTES:\n\n")
                file.write(f"![[include/{db}/{db_schema}/{table}_user_notes.md]]\n\n")



NameError: name 'deps' is not defined